In [1]:
from pathlib import Path
from typing import List, Dict
import yaml
import pandas as pd
from PIL import Image
import os
import numpy as np
import matplotlib.pyplot as plt

In [2]:
SUPPORTED_FORMATS = [".jpg", ".jpeg", ".png", ".bmp", ".tiff", ".gif", ".webp"]

In [3]:
def scan_images_metadata(root_dir: str) -> List[Dict]:
    image_files = []
    root = Path(root_dir)

    for filepath in root.rglob("*"):
        if filepath.suffix.lower() in SUPPORTED_FORMATS:
            try:
                stat = filepath.stat()
                image_files.append({
                    "path": str(filepath.resolve()),
                    "name": filepath.name,
                    "ext": filepath.suffix.lower(),
                    "size": stat.st_size,
                    "modified": stat.st_mtime,
                })
            except Exception as e:
                print(f"Skipping {filepath}: {e}")
    
    return image_files

In [4]:
# Load the gallery path from config.yaml
with open("../config.yaml", "r") as f:
    config = yaml.safe_load(f)

SOURCE_DIR = Path(config["gallery_path"])

In [5]:
photos = scan_images_metadata(SOURCE_DIR)
print(f"Found {len(photos)} images")

Found 1816 images


In [7]:
df = pd.DataFrame(photos)
df.to_parquet("../data/scanned_images_metadata.parquet")

In [8]:
total_size_bytes = df['size'].sum()
total_size_gb = total_size_bytes / (1024 ** 3)
print(f"Gallery size (on disk): {total_size_gb:.2f} GB")

Gallery size (on disk): 2.88 GB


> - ### Preprocess sample images

In [9]:
def preprocess_image(path: str, size=(224, 224)) -> np.ndarray:
    try:
        img = Image.open(path).convert("RGB")
        img = img.resize(size)
        return np.array(img)
    except Exception as e:
        print(f"Failed to preprocess {path}: {e}")
        return None

In [10]:
def preprocess_batch(file_list: List[str], size=(224, 224)) -> List[np.ndarray]:
    results = []
    for path in file_list:
        preprocessed = preprocess_image(path, size)
        if preprocessed is not None:
            results.append(preprocessed)
    return results

In [11]:
def show_images(images, columns=5, size=(15, 5)):
    n_images = len(images)
    rows = (n_images + columns - 1) // columns
    plt.figure(figsize=size)
    
    for i, img in enumerate(images):
        plt.subplot(rows, columns, i + 1)
        plt.imshow(img)
        plt.axis("off")
    
    plt.tight_layout()
    plt.show()

In [13]:
# Load your previously scanned metadata
df = pd.read_parquet("../data/scanned_images_metadata.parquet")
df = df[df["size"] > 0].copy()
df.shape

(1816, 5)

In [14]:
# Select a small batch to test
sample_paths = df["path"].head(100).tolist()

images = preprocess_batch(sample_paths)
print(f"Successfully preprocessed {len(images)} images")

Successfully preprocessed 100 images


> - ### Generate Embeddings with CLIP

In [15]:
from transformers import CLIPProcessor, CLIPModel
from PIL import Image
import torch
from tqdm import tqdm

In [16]:
from collections import defaultdict
import shutil

In [17]:
# Load pre-trained CLIP model & processor
device = "cuda" if torch.cuda.is_available() else "cpu"
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to(device)
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

In [18]:
def get_image_embeddings(images: list) -> torch.Tensor:
    # Convert list of numpy arrays to list of PIL Images
    pil_images = [Image.fromarray(img) for img in images]
    
    # Preprocess and tokenize
    inputs = processor(images=pil_images, return_tensors="pt", padding=True).to(device)
    
    # Get image embeddings
    with torch.no_grad():
        outputs = model.get_image_features(**inputs)
    
    return outputs.cpu()

In [19]:
embeddings = get_image_embeddings(images)
print(f"Embedding shape: {embeddings.shape}")

c:\users\ynany\appdata\local\programs\python\python38\lib\site-packages\transformers\models\clip\modeling_clip.py:540: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


Embedding shape: torch.Size([100, 512])


In [20]:
def compute_similarity_matrix(embeddings: torch.Tensor) -> np.ndarray:
    embeddings = torch.nn.functional.normalize(embeddings, p=2, dim=1)
    sim_matrix = embeddings @ embeddings.T
    return sim_matrix.cpu().numpy()

In [21]:
def find_duplicate_pairs(sim_matrix: np.ndarray, paths: list, threshold) -> pd.DataFrame:
    n = sim_matrix.shape[0]
    pairs = []

    for i in range(n):
        for j in range(i + 1, n):
            sim_score = sim_matrix[i, j]
            if sim_score >= threshold:
                pairs.append({
                    "img1": paths[i],
                    "img2": paths[j],
                    "similarity": sim_score
                })

    return pd.DataFrame(pairs)

In [23]:
sim_matrix = compute_similarity_matrix(embeddings)
duplicate_df = find_duplicate_pairs(sim_matrix, sample_paths, threshold=0.95)
print(duplicate_df.shape)

(29, 3)


In [25]:
def group_duplicates(duplicate_df):
    parent = {}

    def find(x):
        if parent.get(x, x) != x:
            parent[x] = find(parent[x])
        return parent.get(x, x)

    def union(x, y):
        px, py = find(x), find(y)
        if px != py:
            parent[py] = px

    # Create union-find structure
    for _, row in duplicate_df.iterrows():
        union(row["img1"], row["img2"])

    # Group images by their root parent
    groups = defaultdict(list)
    for img in set(duplicate_df["img1"]).union(duplicate_df["img2"]):
        root = find(img)
        groups[root].append(img)

    return list(groups.values())


In [26]:
duplicate_groups = group_duplicates(duplicate_df)
print(f"Found {len(duplicate_groups)} duplicate groups")

Found 17 duplicate groups


In [27]:
# Show group sizes
total=0
for i, group in enumerate(duplicate_groups):
    total+=len(group)
    print(f"Group {i+1}: {len(group)} images")

Group 1: 5 images
Group 2: 2 images
Group 3: 2 images
Group 4: 4 images
Group 5: 3 images
Group 6: 3 images
Group 7: 2 images
Group 8: 2 images
Group 9: 2 images
Group 10: 2 images
Group 11: 2 images
Group 12: 2 images
Group 13: 2 images
Group 14: 2 images
Group 15: 2 images
Group 16: 3 images
Group 17: 2 images


In [28]:
total

42

> - ### Move or Copy Duplicates

In [31]:
def move_or_copy_duplicates(duplicate_groups, action="copy", output_dir="../data/sample_duplicates"):
    
    assert action in ["copy", "move"], "Action must be 'copy' or 'move'"
    output_dir = Path(output_dir)
    output_dir.mkdir(parents=True, exist_ok=True)

    moved_files = []

    for group in duplicate_groups:
        if len(group) < 2:
            continue

        keep = group[0]
        to_move = group[1:]

        for path in to_move:
            src = Path(path)
            filename = src.name
            dst = output_dir / filename

            # Avoids overwriting
            i = 1
            while dst.exists():
                dst = output_dir / f"{src.stem}_{i}{src.suffix}"
                i += 1

            try:
                if action == "copy":
                    shutil.copy2(src, dst)
                else:
                    shutil.move(src, dst)

                moved_files.append({"original": str(src), "duplicate": str(dst)})
            except Exception as e:
                print(f"Failed to {action} {src} → {dst}: {e}")

    return moved_files

In [32]:
log = move_or_copy_duplicates(duplicate_groups, action="move", output_dir="../data/sample_duplicates")
len(log)

17